## Import Dependencies 

In [1]:
import sys

# sys.path.insert(0, '../MIR')

In [2]:
from src.utils.construct_positional_indexes import construct_positional_indexes
from src.utils.load_index import load_index
from src.models.TextPreparer import TextPreparer
from src.enums import *

In [3]:
manager = construct_positional_indexes()
manager.save_index("manager.pickle")

100%|██████████| 1572/1572 [00:14<00:00, 110.47it/s]


In [3]:
manager = load_index("manager.pickle")

In [4]:
manager.corpus_index.get_posting_list("با")[0].get_positions(Fields.TEXT)

[278, 313, 330, 477]

In [4]:
word = "آقو"

In [6]:
manager.correct_word(word)

'آقوس'

In [13]:
max(a, key=lambda item: a[item]/(len(item) + 1 - a[item] + len(word) + 1))

'آقوس'

In [4]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [5]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [5]:
manager.search("نیمه شعبان")

[5619,
 3666,
 5236,
 6208,
 7123,
 6177,
 3665,
 6791,
 3360,
 6475,
 4438,
 7101,
 6626,
 6548,
 4335]

In [5]:
manager.add_document_to_indexes("./data/Add.xml", 7157)

In [9]:
manager.search({Fields.TITLE: '"آقا علیرضا"'})

[]

In [8]:
manager.corpus_index.index["آقو"].doc_frequency

KeyError: 'آقو'

In [8]:
manager.delete_document_from_indexes("./data/Add.xml", 7157)

In [10]:
print(manager.corpus_index.index["آقا"].posting_list[-1])


        Document ID: 7153
        Title Positions: []
        Text Positions: [1049, 1211, 1306]
        


In [11]:
manager.corpus_index.index["آقا"].doc_frequency

{<Fields.TITLE: 'title'>: 0, <Fields.TEXT: 'text'>: 61}

In [14]:
import numpy as np

from src.utils.read_queries import read_queries
from src.metrics import (
    f_measure,
    precision,
    r_precision,
    recall,
    ndcg_at_k,
    average_precision,
    evaluate_search_engine,
)

In [15]:
evaluate_search_engine(manager)

No document satisfies the current phrasal search!


/home/alireza/PycharmProjects/MIR/src/metrics/__init__.py:14: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(relevant) / len(relevant)
/home/alireza/anaconda3/envs/skyrim/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3257: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/alireza/anaconda3/envs/skyrim/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No document satisfies the current phrasal search!


/home/alireza/PycharmProjects/MIR/src/metrics/__init__.py:24: RuntimeWarning: invalid value encountered in double_scalars
  return 2 * precision_val * recall_val / (precision_val + recall_val)


No document satisfies the current phrasal search!
No document satisfies the current phrasal search!


(0.07174799393334863, 0.08944342206165525, nan, nan)

In [6]:
MAX_RETRIEVED = 15

In [3]:
def evaluate_search_engine(query_id="all"):
    queries, relevants = read_queries(query_id)
    results_r_precision = []
    results_ndcg = []
    results_f_measure = []
    results_map = []
    for query, query_relevants in zip(queries, relevants):
        num_relevant_docs = len(query_relevants)
        retrieved_docs = manager.search(query, k=num_relevant_docs)
        relevance = []
        for retrieved_doc in retrieved_docs:
            if retrieved_doc in query_relevants:
                relevance.append(1)
            else:
                relevance.append(0)
        results_r_precision.append(r_precision(relevance, num_relevant_docs))
        results_ndcg.append(ndcg_at_k(relevance, num_relevant_docs))
        results_f_measure.append(f_measure(relevance, num_relevant_docs))
        results_map.append(average_precision(relevance))
    return (
        np.mean(results_r_precision),
        np.mean(results_ndcg),
        np.mean(results_f_measure),
        np.mean(results_map),
    )

In [7]:
evaluate_search_engine()

(0.6723282130404556,
 0.7058278702806886,
 0.6723282130404556,
 0.8153515319937519)